In [50]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import geopandas as gpd

import warnings
warnings.filterwarnings("ignore")

In [5]:
# file path to raw data 
file_path = 'raw data/pp-complete.csv'

# define columns names
# define column names 
column_names = ['transaction_id','price','transfer_date',
              'postcode', 'property_type','new_build_flag',
              'duration','primary_address', 'secondary_address',
              'street','locality','city','district',
              'country','ppd_category','record_status']

# specify data types
dtypes = dict.fromkeys(column_names, 'string')
dtypes.update({'price': 'int64'})

# specify dates to parse
parse_dates = ['transfer_date']

# define columns to keep
usecols = [col for col in column_names if col != 'transaction_id']

ddf = dd.read_csv(file_path, header=None, names=column_names, dtype=dtypes, parse_dates=parse_dates, usecols=usecols)

In [6]:
# compute ddf to pandas dataframe
raw_data_df = ddf.compute()

In [7]:
# postcode = null
raw_data_df[raw_data_df['postcode'].isnull()]

,price,transfer_date,postcode,property_type,new_build_flag,duration,primary_address,secondary_address,street,locality,city,district,country,ppd_category,record_status
2017,155000,1995-08-31,<NA>,T,N,F,46,UNITS 1 AND 2,HIGH STREET,TARPORLEY,TARPORLEY,VALE ROYAL,CHESHIRE,A,A
2999,60000,1995-10-16,<NA>,D,N,F,3,<NA>,PILGRIMS LANE,CHILHAM,CANTERBURY,ASHFORD,KENT,A,A
4873,19500,1995-05-03,<NA>,D,N,F,82,<NA>,BROOKSBY LANE,CLIFTON,NOTTINGHAM,NOTTINGHAM,NOTTINGHAMSHIRE,A,A
9047,32000,1995-10-13,<NA>,T,N,F,65,<NA>,PENVENTON TERRACE,FOUR LANES,REDRUTH,KERRIER,CORNWALL,A,A
10615,23500,1995-04-03,<NA>,D,N,F,LAND ON THE NORTH WESTERN SIDE OF,<NA>,WOODBOROUGH ROAD,MAPPERLEY,NOTTINGHAM,NOTTINGHAM,NOTTINGHAMSHIRE,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364168,350000,2024-01-10,<NA>,O,N,F,WHEAL GREY,<NA>,TRESOWES,ASHTON,HELSTON,CORNWALL,CORNWALL,B,A
364183,141000,2024-01-10,<NA>,O,N,L,SITE 7,<NA>,WILLOWHOLME INDUSTRIAL ESTATE,<NA>,CARLISLE,CUMBERLAND,CUMBERLAND,B,A
364208,173500,2024-01-15,<NA>,O,N,F,GARAGE,<NA>,BROOKSIDE AVENUE,RAINFORD,ST HELENS,ST HELENS,MERSEYSIDE,B,A
365564,7642758,2024-01-02,<NA>,O,N,F,BEAU NASH LANE,FLATS 1-2,<NA>,<NA>,TUNBRIDGE WELLS,TUNBRIDGE WELLS,KENT,B,A


In [8]:
# drop rows with no postcode
clean_data_df = raw_data_df.dropna(subset=['postcode'])

# drop rows with property type 'O'
clean_data_df = clean_data_df.loc[clean_data_df['property_type']!='o']

# sort by date
clean_data_df.sort_values(by=['transfer_date'], inplace=True, ignore_index=True)

In [9]:
clean_data_df.head()

,price,transfer_date,postcode,property_type,new_build_flag,duration,primary_address,secondary_address,street,locality,city,district,country,ppd_category,record_status
0,36000,1995-01-01,GL5 4LU,S,N,F,43,<NA>,MOSLEY ROAD,STROUD,STROUD,STROUD,GLOUCESTERSHIRE,A,A
1,50000,1995-01-01,CT19 6HL,S,N,F,92,<NA>,JOYES ROAD,FOLKESTONE,FOLKESTONE,SHEPWAY,KENT,A,A
2,59950,1995-01-01,NE6 4SU,S,N,F,68,<NA>,APPLETREE GARDENS,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE,TYNE AND WEAR,A,A
3,50000,1995-01-01,BS8 1BY,T,N,F,6,<NA>,MERIDIAN VALE,BRISTOL,BRISTOL,CITY OF BRISTOL,CITY OF BRISTOL,A,A
4,11500,1995-01-01,BB9 9RQ,T,N,F,80,<NA>,FIR STREET,NELSON,NELSON,PENDLE,LANCASHIRE,A,A


In [10]:
# info 
clean_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28873213 entries, 0 to 28873212
Data columns (total 15 columns):
 #   Column             Dtype         
---  ------             -----         
 0   price              int64         
 1   transfer_date      datetime64[ns]
 2   postcode           string        
 3   property_type      string        
 4   new_build_flag     string        
 5   duration           string        
 6   primary_address    string        
 7   secondary_address  string        
 8   street             string        
 9   locality           string        
 10  city               string        
 11  district           string        
 12  country            string        
 13  ppd_category       string        
 14  record_status      string        
dtypes: datetime64[ns](1), int64(1), string(13)
memory usage: 5.1 GB


In [11]:
# postcode_sector column
clean_data_df['postcode_sector'] = clean_data_df['postcode'].apply(lambda x: x[:x.find(' ')+ 2])

In [12]:
clean_data_df.head()

,price,transfer_date,postcode,property_type,new_build_flag,duration,primary_address,secondary_address,street,locality,city,district,country,ppd_category,record_status,postcode_sector
0,36000,1995-01-01,GL5 4LU,S,N,F,43,<NA>,MOSLEY ROAD,STROUD,STROUD,STROUD,GLOUCESTERSHIRE,A,A,GL5 4
1,50000,1995-01-01,CT19 6HL,S,N,F,92,<NA>,JOYES ROAD,FOLKESTONE,FOLKESTONE,SHEPWAY,KENT,A,A,CT19 6
2,59950,1995-01-01,NE6 4SU,S,N,F,68,<NA>,APPLETREE GARDENS,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE,TYNE AND WEAR,A,A,NE6 4
3,50000,1995-01-01,BS8 1BY,T,N,F,6,<NA>,MERIDIAN VALE,BRISTOL,BRISTOL,CITY OF BRISTOL,CITY OF BRISTOL,A,A,BS8 1
4,11500,1995-01-01,BB9 9RQ,T,N,F,80,<NA>,FIR STREET,NELSON,NELSON,PENDLE,LANCASHIRE,A,A,BB9 9


#### Add Region Column ####

In [13]:
# load regions by postcodes 
regions_df = pd.read_csv('postcode_region/postcode_areas.csv')

# create dictionary from df 
postcode_region_dict = dict(zip(regions_df['postcode_prefix'], regions_df['region']))

# Add postcode prefix column to main df
clean_data_df['postcode_prefix'] = clean_data_df['postcode'].str.extract(r'^([A-Z]+)')

# map postcode prefix to the region using the dictionary
clean_data_df['region'] = clean_data_df['postcode_prefix'].map(postcode_region_dict)

# drop the intermidiate row
clean_data_df.drop(columns=['postcode_prefix'], inplace=True)

# exclude sales in Scotland as limited data available 
clean_data_df = clean_data_df.loc[clean_data_df['region'] != 'Scotland']

#### Add Year Column ####

In [14]:
# extract year from transfer data 
clean_data_df['year'] = clean_data_df['transfer_date'].dt.year

# extract month
clean_data_df['month'] = clean_data_df['transfer_date'].dt.month

#### Create GeoJSON Files ####

##### Convert Shapefiles to GeoJSON

In [15]:
shpfile = gpd.read_file('shapefiles/Sectors.shp')
shpfile.to_file('GeoJSON/allsectors.geojson', driver='GeoJSON')

Create GeoJSON File for each Region 

In [16]:
# load postcode prefix CSV
postcode_prefixes_df = pd.read_csv('postcode_region/postcode_areas.csv')

# load the geojson file containing all postcode sector boundaries
geojson_path = 'GeoJson/allsectors.geojson'

# read the geojson
postcode_sectors_gdf = gpd.read_file(geojson_path)

In [17]:
# create empty dictionary to store 
region_geojson_dict = {}

# extract postcode prefix from 'name' property within geodf
postcode_sectors_gdf['postcode_prefix'] = pd.Series(postcode_sectors_gdf['name']).str.extract(r'^([A-Z]+)')

# merge postcode prefixes df to get regions 
merged_df = pd.merge(postcode_sectors_gdf, postcode_prefixes_df, how='left', on= 'postcode_prefix')

In [23]:
# iterate over each unique region 
for region in merged_df['region'].unique():
    # filter df for the current region
    regions_df = merged_df[merged_df['region'] == region].copy()

    # create a geo df for the current region
    regions_gdf = gpd.GeoDataFrame(regions_df, geometry='geometry',crs=postcode_sectors_gdf.crs)

    # store the geo df in the dictionary under the region key
    region_geojson_dict[region] = regions_gdf

We create a nested dictionary

In [24]:
region_geojson_dict

{'Scotland':          name                                           geometry  \
 0      AB10 1  POLYGON ((-2.11645 57.14656, -2.11655 57.14663...   
 1      AB10 6  MULTIPOLYGON (((-2.12239 57.12887, -2.12279 57...   
 2      AB10 7  POLYGON ((-2.12239 57.12887, -2.12119 57.12972...   
 3      AB11 5  POLYGON ((-2.05528 57.14547, -2.05841 57.14103...   
 4      AB11 6  POLYGON ((-2.09818 57.13769, -2.09803 57.13852...   
 ...       ...                                                ...   
 9583    TD9 9  POLYGON ((-2.75085 55.42979, -2.74868 55.42674...   
 10810   ZE1 0  MULTIPOLYGON (((-1.17282 60.12108, -1.18797 60...   
 10811   ZE1 9  POLYGON ((-1.14734 60.15442, -1.14697 60.15457...   
 10812   ZE2 9  MULTIPOLYGON (((-0.80210 60.40932, -0.80233 60...   
 10813   ZE3 9  MULTIPOLYGON (((-1.38060 59.88701, -1.37423 59...   
 
       postcode_prefix postcode_district    region  
 0                  AB          Aberdeen  Scotland  
 1                  AB          Aberdeen  Scotland  

In [25]:
region_geojson_dict['East England']

,name,geometry,postcode_prefix,postcode_district,region
7238,PE1 1,"POLYGON ((-0.23785 52.57732, -0.23730 52.57691...",PE,Peterborough,East England
7239,PE1 2,"MULTIPOLYGON (((-0.24948 52.59020, -0.24929 52...",PE,Peterborough,East England
7240,PE1 3,"POLYGON ((-0.24948 52.59020, -0.25004 52.59052...",PE,Peterborough,East England
7241,PE1 4,"POLYGON ((-0.21524 52.58909, -0.21613 52.58943...",PE,Peterborough,East England
7242,PE1 5,"POLYGON ((-0.21524 52.58909, -0.21386 52.59030...",PE,Peterborough,East England
...,...,...,...,...,...
7375,PE9 1,"POLYGON ((-0.49609 52.65722, -0.49617 52.65733...",PE,Peterborough,East England
7376,PE9 2,"POLYGON ((-0.51436 52.65164, -0.51274 52.65597...",PE,Peterborough,East England
7377,PE9 3,"POLYGON ((-0.51436 52.65164, -0.51227 52.65106...",PE,Peterborough,East England
7378,PE9 4,"POLYGON ((-0.38126 52.65034, -0.38583 52.65018...",PE,Peterborough,East England


Export the GeoJSON Files 

In [26]:
for region, regions_gdf in region_geojson_dict.items():
    region_geojson_path = f'GeoJSON/regions/{region}_postcode_sectors.geojson'
    regions_gdf.to_file(region_geojson_path, driver='GeoJSON')

Process Data

Calculate average price for each postcode by year 

In [27]:
# group by region, postcode sector and year then calculate avg price 
grouped_df = clean_data_df.groupby(['region','postcode_sector','year']).agg({'price': ['mean', 'count']}).reset_index()

grouped_df.columns = ['region','postcode_sector','year','avg_price','volume']

# round the avg price to nearest thousand and convert to int
grouped_df['avg_price'] = grouped_df['avg_price'].round(-3).astype(int)

In [28]:
grouped_df.head()

,region,postcode_sector,year,avg_price,volume
0,East England,PE1 1,1995,46000,11
1,East England,PE1 1,1996,39000,10
2,East England,PE1 1,1997,52000,15
3,East England,PE1 1,1998,49000,21
4,East England,PE1 1,1999,77000,20


In [30]:
# loop though each unique year and save CSV for each
for year in grouped_df['year'].unique():
    # filter df for current year
    year_df = grouped_df[grouped_df['year'] == year]

    # create file path for current year CSV 
    csv_file_path = f'processed_data/average_price_by_year/region_data_{year}.csv'

    # save the df to csv 
    year_df.to_csv(csv_file_path, index=False)

Calculate average price for each region by year

In [32]:
# group by year, region, property type and calculate average price
region_grouped_df = clean_data_df.groupby(['year','region','property_type']).agg(avg_price=('price','mean')).round({'avg_price': -3}).astype({'avg_price': int}).reset_index()

# calculate sales volume for each group
region_grouped_df['volume'] = clean_data_df.groupby(['year','region','property_type']).size().reset_index(name='volume')['volume']

In [33]:
region_grouped_df.head()

,year,region,property_type,avg_price,volume
0,1995,East England,D,72000,6173
1,1995,East England,F,34000,598
2,1995,East England,S,45000,4092
3,1995,East England,T,37000,3131
4,1995,East Midlands,D,78000,21596


In [34]:
# save df to csv file 
region_avg_csv_file_path = 'processed_data/region_avg_price/region_avg_prices.csv'

region_grouped_df.to_csv(region_avg_csv_file_path, index=False)

Calculate delta % year-on-year

In [35]:
# group by region and postcode sector, then calculate delta
delta_df = grouped_df.copy()
delta_df['delta'] = grouped_df.groupby(['region','postcode_sector'])['avg_price'].pct_change() * 100

# set the delta to 0 for first year
delta_df.loc[delta_df['year'] == delta_df['year'].min(), 'delta'] = 0

In [36]:
delta_df.head()

,region,postcode_sector,year,avg_price,volume,delta
0,East England,PE1 1,1995,46000,11,0.000000
1,East England,PE1 1,1996,39000,10,-15.217391
2,East England,PE1 1,1997,52000,15,33.333333
3,East England,PE1 1,1998,49000,21,-5.769231
4,East England,PE1 1,1999,77000,20,57.142857


In [48]:
delta_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240555 entries, 0 to 241424
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   region           240555 non-null  object 
 1   postcode_sector  240555 non-null  object 
 2   year             240555 non-null  int32  
 3   avg_price        240555 non-null  int32  
 4   volume           240555 non-null  int64  
 5   delta            240555 non-null  float64
dtypes: float64(1), int32(2), int64(1), object(2)
memory usage: 11.0+ MB


In [51]:
# drop any rows with a null delta
delta_df = delta_df.dropna(subset=['delta'])

# Fill non-finite values with 0
delta_df['delta'] = delta_df['delta'].replace([np.inf, -np.inf, np.nan], 0)

# round values to nearest int
delta_df['delta'] = delta_df['delta'].round().astype(int)

In [52]:
delta_df.head()

,region,postcode_sector,year,avg_price,volume,delta
0,East England,PE1 1,1995,46000,11,0
1,East England,PE1 1,1996,39000,10,-15
2,East England,PE1 1,1997,52000,15,33
3,East England,PE1 1,1998,49000,21,-6
4,East England,PE1 1,1999,77000,20,57


In [61]:
# iterate over each year and create a csv for each
for year in delta_df['year'].unique():
    # filter rows for the current year
    year_df = delta_df[delta_df['year'] == year]

    # create csv for the current year 
    csv_file_path = f'processed_data/avg_price_delta/avg_prices_delta_{year}.csv'
    year_df.to_csv(csv_file_path, index=False)

In [62]:
delta_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240555 entries, 0 to 241424
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   region           240555 non-null  object
 1   postcode_sector  240555 non-null  object
 2   year             240555 non-null  int32 
 3   avg_price        240555 non-null  int32 
 4   volume           240555 non-null  int64 
 5   delta            240555 non-null  int32 
dtypes: int32(3), int64(1), object(2)
memory usage: 10.1+ MB


Create volume by month for each year 

In [63]:
# group by region, postcode_sector, year and month then calculate count of sales
volume_df = clean_data_df.groupby(['region','postcode_sector','year','month']).agg({'price':'count'}).reset_index()

# rename volumes for clarity
volume_df.columns = ['regions','postcode_sector','year','month','volume']

In [65]:
# loop throug each year and create separate csvs
for year in volume_df['year'].unique():
    # filter df for current year
    year_df = volume_df[volume_df['year'] == year]

    # group by region, month then calculate volume
    total_volume_df = year_df.groupby(['regions','month']).agg({'volume':'sum'}).reset_index()

    # create csv from df 
    csv_file_path = f'processed_data/volume_by_year/region_total_volume_{year}.csv'
    total_volume_df.to_csv(csv_file_path, index=False)